In [38]:
import smtplib 
import pandas as pd 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText


def set_email_header(message, email_config, email_to):
    message['From'] = email_config['From']
    message['Bcc'] = email_config['Bcc']
    message['Subject'] = email_config['Subject']
    message['To'] = email_to
    
def read_auth_tokens(fname):
    fauth = open(fname, 'r')
    user, password = fauth.readlines()
    fauth.close()
    return user.strip(), password.strip()

def getEmailContent(row,headers):
    #create the body of the email 
    text = """
    <head>
     <style>
    table {
        margin-left: auto;
        margin-right: auto;
    }
    </style>
    <style>
    table, th, td {
        border: 1px solid black;
    }
    </style>
    </head>
    """
    text += f"""
    Cordial saludo, <strong>{row[2]} {row[3]}</strong>,<br>
    Este es tu registro de asistencias y notas hasta la fecha:<br><br>
    <table>
    <tbody>
    """    
    
    #add header titles    
    for index, label in enumerate(headers[5:]):
        text += "<tr style='text-align:center'>"
        text += f"<th>{label}</th><td>{row[index+5]}</td>"
        text += "</tr>"
        
    text += "</tbody>"
    text += "</table>"
    text += """
    <br><br>
    Si tiene alguna reclamación por favor <strong>responder a este mismo correo</strong>, especificar en cuál nota o asistencia, y <strong>adjuntar las evidencias</strong> que sustenten la reclamación. Muchas gracias por su atención. 
    """
    return text



In [1]:
EMAIL_CONFIG = {'From':"your_email",
                'Bcc':"optional_emnail",
                'Subject':"your_subject"}

#data read
data = pd.read_excel("notas.xlsx")
NROWS, NCOLS = data.shape
headers = data.columns
print(headers[1])

#smtp server, you have to create a file .txt with your user and password
s = smtplib.SMTP(host = 'smtp.gmail.com', port = 587)
s.starttls()
CREDENTIALS_USER, CREDENTIALS_PASS = read_auth_tokens('credentials.txt')
s.login(CREDENTIALS_USER, CREDENTIALS_PASS)

#loop over each row
for index, row in data.iterrows():
    print(f"Sending email to: {row[2]} {row[3]}")
    msg = MIMEMultipart()
    set_email_header(msg, EMAIL_CONFIG, email_to = row[4])
    msg.attach(MIMEText(getEmailContent(row,headers), 'html'))
    print(getEmailContent(row,headers))
    s.send_message(msg)
    del msg    
s.quit()

NameError: name 'pd' is not defined